#  Puzzle dos Colares
## Projeto nº 1
### Introdução à Inteligência Artificial edição 2020/21



<img src="finalDoisColares.PNG" alt="Drawing" style="width: 250px;"/>

## Grupo: XX

### Elementos do Grupo

Nome: Rui Fartaria

Número: 18752

Nome: António Fróis

Número: 51050


### Modelização em Python

A construção do puzzle de colares intersectantes é feita de forma incremental, utilizado as seguintes classes:
1. **Bead**: representa uma conta do colar
2. **Necklace**: representa um colar. É modelado por uma lista de instâncias de Beads
3. **IntersectedNecklacesState**: representa um estado do problema. É modelado por uma lista de instâncias de Neclaces.
4. **PuzzleColares**: representa o problema do puzzle de colares. É modelado por uma classe derivada da classe Problem, definida no módulo *searchPlus.py*.

#### Classe Bead

Representa uma conta do colar. Tem apenas um atributo: a côr. O método __str__ foi *overloaded* para mostrar um círculo em que o a côr é codificada usando sequencias *escape* ANSI.

#### Classe Necklace

Representa um colar de contas coloridas, modelado por uma lista de instâncias de *Bead*.

Tem vários métodos utilitários para se obter informação sobre as cores e distribuição de cores e para manusear as contas. São de realçar os métodos:

- **rotateColours(self, direction)**: roda as cores das contas, e não as contas! Este método é muito importante porque é o que vai facilitar a partilha de contas entre colares, permitindo que as cores rodem num colar de forma a ficarem automaticamente actualizadas nos colares adjacentes.


- **__str__**: *overloaded* para mostrar uma lista colorida de circulos, que representam as contas.

#### Classe IntersectedNecklacesState

Representa uma cadeia de colares intersectantes, modelada mor uma lista de instâncias de Necklace.

Os métodos mais importantes são:

- **\_\_init\_\_(self, dimension=2, numBeads=20, initConf=None)**: no construtor são especifcados o número de colares e o tamanho dos mesmos. Pode também ser fornecida uma configuração inicial através de uma lista de listas de inteiros (vêr regras no comentário do método *\_\_init\_\_*). Em primeiro lugar é construida uma lista de instâncias de Necklace independentes. Essas são depois entrelaçadas pegando nos colares em sequência e fazendo com que duas contas do colar da direita sejam substituidas por duas contas do colar da esquerda (vêr código em *_generateRandomConfiguration(self)*). Em seguida são colocadas as cores, aleatoriamente, de acordo com a distribuição de cores compatível com o número e tamanho de colares (vêr regras no comentário do método *\_\_init\_\_*). Finalmente, se tiver sido fornecida uma configuração inicial, esta é aplicada substituindo as cores iniciais (colocadas aleatóriamente). Neste caso, é também feita uma série de testes para verificar se a configuração fornecida está de acordo com o número de colares, o tamanho dos colares e a regra de distribuição de cores.


- **\_\_str\_\_(self)**: devolve uma string em que os aneis são representados em ASCII 2D, como círculos encadeados com contas coloridas. Para isso recorre primeiro a um *screen buffer* de caracteres onde é construida a imagem que irá ser depois *imprimida* linha a linha no ecrã. Esta representação possibilita uma visualização intuitiva do estado e permite identificar facilmente alterações (rotações dos aneis) de estado para estado.


- **\_\_eq\_\_(self, o)**: compara dois estados, utilizando o critério de igualdade de cores (e não de contas). Portanto, dois estados são considerados iguais se a sequência de cores de contas for a mesma nos dois estados.


- **i_am_a_goal_state(self)**: devolve *True* se este estado corresponder a um estado do puzzle resolvido. Ou seja: cada colar tem duas cores completas em sequências ininterruptas. Para melhorar a eficiência, primeiro verifica se todos os colares têm duas cores completas e só depois verifica se estas se encontram dispostas continuamente.


- **listifyed(self)**: devolve uma versão serializada (sob a forma de uma lista de listas de inteiros) do estado, que pode ser usada no construtor de um novo estado no parâmetro *initialConf*.


- **rotateColours(self, iNecklace, direction)**: roda as cores de qualquer colar (especificado em *iNecklace*), com direcção e magnitude definidas pelo parâmetro *direction*. Se *direction* for negativo a rotação é feita no sentido dos ponteiros do relógio. 

#### Classe PuzzleColares

Representa o problema do puzzle de colares. É uma classe derivada da classe Problem, defininida no módulo *searchPlus.py*.

Foram implementados os seguintes métodos:

- **def actions(self, state)**: devolve as acções que podem ser usadas em qualquer estado, com o respectivo custo, sob a forma de uma lista de dicionários. Estão definidas apenas duas acções, com custo unitário: rotação positiva (contrária aos ponteiros do relógio) com magnitude 1 (só roda uma conta) e rotação negativa com a mesma magnitude. Poderiam ser definidas mais acções com magnitudes e custos diferentes, e.g., magnitude 2 e custo 1.8 ou magnitude 3 e custo 1.5, etc.


- **result(self, state, action):**: faz um clone do estado fornecido, utilizando o método *listifyed* de IntersectedNecklacesState. Em seguida aplica a acção ao clone e devolve o estado resultante.


- **path_cost(self, c, state1, action, state2)**: devolve o custo acumulado (assumindo um custo prévio *c*) da transição do estado *state1* para *state2* através da acção *action*.

- **goal_test(self, state)**: faz uso do método *i_am_a_goal_state* de *state* para verificar se é um estado final do puzzle resolvido.


In [30]:

import random
from searchPlus import *

class Bead(object):
    
    def __init__(self, colour):
        self._colour = colour

    def getColour(self):
        return self._colour
    
    def setColour(self, colour):
        self._colour = colour

    def __str__(self):
        return "\u001b[38;5;"+str(self._colour)+"mO\u001b[0m"
        #return "\u001b[38;5;"+str(self._colour)+"m"+str(self._colour)+"\u001b[0m"



class Necklace(object):
    
    """ necklace: necklace to clone
        colourBeadsDist: quick and dirty way to initialize a necklace with a 
                         distribution of coloured beads using a dictionary"""
    def __init__(self, necklace=None, colourBeadsDist={1:10,2:9,3:1}):
            
        if necklace != None:
            raise NotImplementedError
        else:
            self._numBeads = sum(colourBeadsDist.values())
            self._beads = [];
            for k in colourBeadsDist.keys():
                for i in range(colourBeadsDist[k]):
                    self._beads.append(Bead(k));


    def randomizeColors(self):
        for i in range(len(self.numBeads)):
            a = random.choice(self._beads)
            b = random.choice(self._beads)
            while a == b:
                b = random.choice(self._beads)
            
    
    def getNumBeads(self):
        return self._numBeads


    def getBead(self, k):
        return self._beads[k]
    
    
    def getBeads(self):
        return self._beads


    def getBeadColours(self):
        return [b.getColour() for b in self._beads]
    
    
    def getBeadColourDistribution(self):
        bcs = self.getBeadColours()
        colourSet = set(bcs)
        cdist =  {}
        for c in colourSet:
            cdist[c] = bcs.count(c)
        return cdist


    def removeBead(self, k):
        if (isinstance(k, int)):
            self._beads.pop(k)
            self._numBeads -= 1
            return
        if (isinstance(k, Bead)):
            self._beads.remove(k)
            self.numBeads -= 1
            return
        raise Exception("k is neither [int] or [Bead]!")


    def appendBead(self, bead):
        self.beads.append(bead)


    def insertBead(self, i, bead):
        self._beads.insert(i, bead)
        self._numBeads += 1


    def replaceBead(self, i, bead):
        self.removeBead(i)
        self.insertBead(i, bead)


    """ Rotate colours in given direction (negative = clockwise)
        direction: integer giving magnitude of rotation"""
    def rotateColours(self, direction):
        if (direction < 0):
            for k in range(-direction):
                colours = [b.getColour() for b in self._beads]
                colours = colours[1:self._numBeads]+[colours[0]]
                for i in range(self._numBeads):
                    self._beads[i].setColour(colours[i])
        else:
            for k in range(direction):
                colours = [b.getColour() for b in self._beads]
                colours = [colours[-1]] + colours[0:self._numBeads-1]
                for i in range(self._numBeads):
                    self._beads[i].setColour(colours[i])


    def __str__(self):
        return "".join([str(x) for x in self._beads])+"\u001b[0m"



class IntersectedNecklacesState(object):
    
    """ dimension: number of necklaces
        numBeads: number of beads in each necklace. Must be divisible by 4
        initConf: list of lists of integers (colour codes) according to following rules
                1. each sub-list represents a necklace
                2. necklaces have common bead colours (shared beads) as follows:
                    2.1 left necklace highest shared bead index is numBeads/2+insersection+1
                    2.1 right necklace lowest shared bead index is 0
                3. total number of colours defined in necklaces set is 2*dimension
                4. color distribution is numbeads/2 for two colours and numbeads/2-1 for the remaining ones
                Example for two rings with 20 beads:
                [[2,1,1,1,1,1,1,1,1,<3>,2,2,2,<2>,2,2,2,2,2,2],
                 [<2>,3,3,3,<3>,3,3,3,3,3,,3,4,4,4,4,4,4,4,4,4]]
                shared beads are signaled for convenience """
                
    def __init__(self, dimension=2, numBeads=20, initConf=None):

        self._dimension = dimension
        self._numBeads = numBeads
        self._intersection = int(numBeads/4) - 2
            
        self._generateRandomConfiguration()
        
        if initConf != None:
            
            # test initConf
            if len(initConf) != self._dimension:
                raise Exception("Initial configuration dimension different from puzzle dimension")
            for l in initConf:
                if len(l) != self._numBeads:
                    raise Exception("Wrong number of beads at "+str(l))
            
            # apply initConf
            for i in range(self._dimension):
                clist = initConf[i]
                necklace = self._necklaces[i]
                for j in range(self._numBeads):
                    necklace.getBead(j).setColour(clist[j])
            
            # further tests on initConf
            cdist = self.getColourDistribution()
            if (len(cdist) != 2*self._dimension):
                raise Exception("Wrong number of colours: "+str(cdist))
            if (len([c for c in cdist.values() if c == int(self._numBeads/2)]) != 2):
                raise Exception("Wrong colour distribution: "+str(cdist))
            if (len([c for c in cdist.values() if c == int(self._numBeads/2)-1]) != 2*self._dimension-2):
                raise Exception("Wrong colour distribution: "+str(cdist))


    def _generateRandomConfiguration(self):
            # generate independent necklaces
            self._necklaces = [Necklace(colourBeadsDist={1:self._numBeads}) for i in range(self._dimension)]
            
            # intersect necklaces
            for i in range(1,len(self._necklaces)):
                leftNecklace = self._necklaces[i-1]
                rightNecklace = self._necklaces[i]
                leftNecklace_k = int(self._numBeads/2) + self._intersection + 1
                leftNecklace_j = leftNecklace_k - self._intersection - 1
                rightNecklace_k = 0
                rightNecklace_j = self._intersection + 1
                rightNecklace.replaceBead(rightNecklace_j, leftNecklace.getBead(leftNecklace_j))
                rightNecklace.replaceBead(rightNecklace_k, leftNecklace.getBead(leftNecklace_k))
            
            # randomize colours
            numColours = self._dimension * 2
            cdim = int(self._numBeads / 2)
            colours = [1 for i in range(cdim)]
            colours += [2 for i in range(cdim)]
            cdim -= 1
            for j in range(3, numColours+1):
                colours += [j for i in range(cdim)]
            
            beads = []
            for necklace in self._necklaces:
                for b in necklace.getBeads():
                    if b not in beads:
                        beads.append(b)
            beads = list(beads)
            while len(colours) > 0:
                c = random.choice(colours)
                colours.remove(c)
                b = beads.pop()
                b.setColour(c)


    def getNecklaces(self):
        return self._necklaces


    def rotateColours(self, iNecklace, direction):
        self._necklaces[iNecklace].rotateColours(direction)


    def getOrderedBeads(self):
        orderedBeads = []
        for necklace in self._necklaces:
            for b in necklace.getBeads():
                if b not in orderedBeads:
                    orderedBeads.append(b)
        return orderedBeads


    def getOrderedBeadColours(self):
        return [b.getColour() for b in self.getOrderedBeads()]


    def getColourDistribution(self):
        allBeadColours = self.getOrderedBeadColours()
        colourSet = set(allBeadColours)
        cdist = {}
        for c in colourSet:
            cdist[c] = allBeadColours.count(c)
        return cdist


    def i_am_a_goal_state(self):
        """Goal is attained when each necklace has two colours condensed 
        in full sequences (complete set)."""
        # get colour distributions
        scdist = self.getColourDistribution()
        for necklace in self.getNecklaces():
            ncdist = necklace.getBeadColourDistribution()
            # must have at least two complete colours
            completeColours = [c for c in ncdist.keys() if scdist[c] == ncdist[c]]
            if len(completeColours) != 2:
                return False
                    
        # all have two complete colours
        # now lets test for full sequences
        for necklace in self.getNecklaces():
            nbColours = necklace.getBeadColours()
            ncdist = necklace.getBeadColourDistribution()
            completeColours = [c for c in ncdist.keys() if scdist[c] == ncdist[c]]
            for c in completeColours:
                ic = nbColours.index(c)
                ccount = 1
                niter = 0
                iup = ic
                idown = ic
                while (nbColours[iup] == c or nbColours[idown] == c):
                    niter += 1
                    iup = (ic + niter) % len(nbColours)
                    idown = (ic - niter) % len(nbColours)
                    if nbColours[iup] == c:
                        ccount += 1
                    if nbColours[idown] == c:
                        ccount += 1
                if ccount != scdist[c]:
                    return False
        # all have two complete colours in full sequence
        return True


    def __str__(self):
        height = self._intersection + 2
        length = int((self._numBeads - 2*height) / 2)
        scrLength = length + 1
        fullLength = scrLength * self._dimension + 4
        
        # blank screen buffer
        scr = [ [" " for i in range(fullLength)] for j in range(height+2) ]
        
        for i in range(self._dimension):
            necklace = self._necklaces[i]
            for j in range(self._numBeads):
                bead = necklace.getBead(j)
                # transform from (i,j) to scr (<l>ine, <c>olumn)
                if (j < 1):
                    l = 1 + j
                    c = i * scrLength + 1
                elif (j < height-1):
                    l = 1 + j
                    c = i * scrLength
                elif (j < height):
                    l = height
                    c = i * scrLength + 1
                elif (j < height+length):
                    l = height + 1
                    c = i * scrLength + (j-length) + 2
                elif (j < height+length+1):
                    l = height
                    c = i * scrLength + length + 2
                elif (j < height+length+height-1):
                    l = height - (j - (height+length))
                    c = i * scrLength + length + 3
                elif (j < height+length+height):
                    l = height - (j - (height+length))
                    c = i * scrLength + length + 2
                else:
                    l = 0
                    c = i * scrLength + length - (j - (height+length+height)) + 1
                scr[l][c] = str(bead)
        return "\n".join(["".join(line) for line in scr])
    
    
    """ Only compares colour values, not the bead objetcs themselves"""
    def __eq__(self, o):
        return tuple(self.getOrderedBeadColours()) == tuple(o.getOrderedBeadColours())


    def listifyed(self):
        return [[b.getColour() for b in necklace.getBeads()] for necklace in self._necklaces]



class PuzzleColares(Problem):
    
    def __init__(self, initial, goal=None):
        super().__init__(initial, goal)


    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        return [{'name':'rotate', 'target':i, 'direction':+1, 'cost':1} for i in range(len(state.getNecklaces()))] + \
               [{'name':'rotate', 'target':i, 'direction':-1, 'cost':1} for i in range(len(state.getNecklaces()))]


    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        newState = IntersectedNecklacesState(dimension=len(state.getNecklaces()), \
                        numBeads=state.getNecklaces()[0].getNumBeads(), \
                        initConf=state.listifyed())
        newState.rotateColours(iNecklace=action['target'], direction=action['direction'])
        return newState
    
    
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + action['cost']


    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        return state.i_am_a_goal_state()


    def display(self, state):
        """ Display state """
        print(state)


def exec(p,estado,accoes):
    """ Executa uma sequência de acções a partir do estado
        devolve um par (estado, custo) depois de imprimir
    """
    custo = 0
    for a in accoes:
        seg = p.result(estado,a)
        custo = p.path_cost(custo,estado,a,seg)
        estado = seg
    return (estado,custo)



### Testes base dos estados


In [35]:
random.seed(1234567)

print()
print("********* STATE TESTS ***********")
print()
instate0 = IntersectedNecklacesState(dimension=2, numBeads=20)
instate1 = IntersectedNecklacesState(dimension=2, numBeads=20, initConf=instate0.listifyed())
print("instate0 == instate1: "+str(instate0 == instate1))
print(instate1)
print("rotate 0, +1")
instate1.rotateColours(0,1)
print(instate1)
print("rotate 1, -1")
instate1.rotateColours(1,-1)
print(instate1)
print("rotate 1, +1")
instate1.rotateColours(1,+1)
print(instate1)
print("rotate 0, -1")
instate1.rotateColours(0,-1)
print(instate1)
print("is the same as it started = "+str(instate0 == instate1))
print()
print("Wrong number of necklaces:")
initcl = [[2,1,1,1,1,1,1,1,1,1,3,2,2,2,2,2,2,2,2,2]]
try:
    state = IntersectedNecklacesState(dimension=2, numBeads=20, initConf=initcl)
except Exception as e:
    print(e)
print()
print("Wrong number of beads:")
initcl = [[2,1,1,1,1,1,1,1,1,1,3,2,2,2,2,2,2,2,2,2],
             [2,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4]]
try:
    state = IntersectedNecklacesState(dimension=2, numBeads=20, initConf=initcl)
except Exception as e:
    print(e)
print()
print("Wrong colour distribution:")
initcl = [[2,1,1,1,1,1,1,1,1,1,3,2,2,2,2,2,2,2,2,2],
             [2,5,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4]]
try:
    state = IntersectedNecklacesState(dimension=2, numBeads=20, initConf=initcl)
except Exception as e:
    print(e)

print()
print("Three rings with 20 beads each:")
instate = IntersectedNecklacesState(dimension=3, numBeads=20)
print(instate)

print()
print("Four rings with 32 beads each:")
instate = IntersectedNecklacesState(dimension=6, numBeads=32)
print(instate)


********* STATE TESTS ***********

instate0 == instate1: True
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
rotate 0, +1
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
rotate 1, -1
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
rotate 1, +1
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
rotate 0, -1
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
is the same as it started = True

Wrong number of necklaces:
Initial configuration dimension different from puzzle dimension

Wrong number of beads:
Wrong number of beads at [2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4]

Wrong colour distribution:
Wrong colour distribution: {1:

### Criação de estados e do problema
(Mostrem que o código está a funcionar, construindo instâncias da classe **PuzzleColares**, fazendo o display dos estados, verificando o teste de estado final, gerando as acções para alguns estados, executando acções a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as acções (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de acções está a funcionar bem.)

In [27]:
random.seed(346347)

print()
print("********* PUZZLE TESTS ***********")
print()
instate = IntersectedNecklacesState(dimension=2, numBeads=20)
print("Initial state:")
print(instate)
puzzle = PuzzleColares(instate)
print("Is goal state: "+ str(puzzle.goal_test(instate)))
print()
# using the one from project statement
goalState = [[2,1,1,1,1,1,1,1,1,1,3,2,2,2,2,2,2,2,2,2],
             [2,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4,4]]
instate = IntersectedNecklacesState(dimension=2, numBeads=20, initConf=goalState)
print("Initial state:")
print(instate)
puzzle = PuzzleColares(instate)
print("Is goal state: "+ str(puzzle.goal_test(instate)))
    


********* PUZZLE TESTS ***********

Initial state:
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
Is goal state: False

Initial state:
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
Is goal state: True


### Teste de Procura de Solução
(Quem quiser ir mais além, até pode confirmar que o problema está bem modelizado utilizando alguns dos algoritmos de procura como o **depth-first-search()** ou o **breadth-first-search()**, nas suas versões em árvore ou em grafo; Para isso poderão pegar num estado final, aplicar-lhe uma sequência de algumas acções (desde que não se tenha de esperar demasiado pela solução) e a partir do estado resultante criar um problema que será objecto do algoritmo de procura. Se encontrar uma solução, a solução espelho da sequência usada, então tudo estará ok)

In [26]:
random.seed(346347)

# using the one from project statement
goalState = [[2,1,1,1,1,1,1,1,1,1,3,2,2,2,2,2,2,2,2,2],
             [2,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4,4]]
instate = IntersectedNecklacesState(dimension=2, numBeads=20, initConf=goalState)
print("Initial state:")
print(instate)
puzzle = PuzzleColares(instate)
print("Is goal state: "+ str(puzzle.goal_test(instate)))

print()
actions = [random.choice(puzzle.actions(instate)) for i in range(5)]
print("Actions:")
for action in actions:
    print(action)

result = exec(puzzle, puzzle.initial, actions)
print("Final state:")
print(result[0])
print ("cost = "+str(result[1]))

print()
initStateList = result[0].listifyed()
instate = IntersectedNecklacesState(dimension=2, numBeads=20, initConf=initStateList)
puzzle = PuzzleColares(instate)
print("Initial state:")
print(puzzle.initial)
print()
print("Performing breadth-first search...")
print()
resultNode = breadth_first_tree_search(puzzle)
print("Final state:")
print(resultNode.state)
print("Is goal state: "+ str(puzzle.goal_test(resultNode.state)))
print()
print("Actions performed:")
for a in resultNode.solution():
    print(a)
print ("cost = "+str(resultNode.path_cost))

Initial state:
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
Is goal state: True

Actions:
{'name': 'rotate', 'target': 0, 'direction': -1, 'cost': 1}
{'name': 'rotate', 'target': 0, 'direction': -1, 'cost': 1}
{'name': 'rotate', 'target': 1, 'direction': -1, 'cost': 1}
{'name': 'rotate', 'target': 0, 'direction': 1, 'cost': 1}
{'name': 'rotate', 'target': 1, 'direction': -1, 'cost': 1}
Final state:
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
cost = 5

Initial state:
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   

Performing breadth-first search...

Final state:
  OOOOO OOOOO   
 O     O     O  
O     O O     O 
O     O O     O 
O     O O     O 
 O     O     O  
  OOOOO OOOOO   
Is goal state: True

Actions performed:
{'name': 'rotate', 'target': 1, 'direction': 1, 'cos